# Optimization Modes

This notebook demonstrates different optimization approaches in **flixopt**:

1. **Full Optimization** - Solve the complete problem at once
2. **Segmented Optimization** - Split into overlapping time segments
3. **Aggregated (Clustered) Optimization** - Reduce complexity via time series aggregation

These modes offer trade-offs between solution quality and computation time, essential for large-scale problems.

## Setup

In [ ]:
import pathlib

import pandas as pd
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

## Helper Function

This function extracts solutions from different optimization types for comparison:

In [ ]:
def get_solutions(optimizations: list, variable: str) -> xr.Dataset:
    """Extract a variable from multiple optimizations for comparison."""
    dataarrays = []
    for optimization in optimizations:
        if optimization.name == 'Segmented':
            # SegmentedOptimization requires special handling to remove overlaps
            dataarrays.append(optimization.results.solution_without_overlap(variable).rename(optimization.name))
        else:
            # For Full and Clustered, access solution from the flow_system
            dataarrays.append(optimization.flow_system.solution[variable].rename(optimization.name))
    return xr.merge(dataarrays, join='outer')

## Configuration

Select which optimization modes to run:

In [ ]:
# Which optimizations to run
full = True
segmented = False  # Set True to compare segmented approach
aggregated = False  # Set True to compare aggregated approach

# Segmented optimization parameters
segment_length = 96  # Hours per segment
overlap_length = 1  # Overlap between segments

# Aggregated optimization parameters
clustering_parameters = fx.ClusteringParameters(
    hours_per_period=6,
    nr_of_periods=4,
    fix_storage_flows=False,
    aggregate_data_and_fix_non_binary_vars=True,
    percentage_of_period_freedom=0,
    penalty_of_period_freedom=0,
)
keep_extreme_periods = True
imbalance_penalty = 1e5

## Load Data

Import time series data for one week:

In [ ]:
# Load data from CSV
data_path = pathlib.Path('..') / '..' / '..' / 'examples' / 'resources' / 'Zeitreihen2020.csv'
data_import = pd.read_csv(data_path, index_col=0).sort_index()

# Filter to one week
filtered_data = data_import['2020-01-01':'2020-01-07 23:45:00']
filtered_data.index = pd.to_datetime(filtered_data.index)
timesteps = filtered_data.index

print(f'Time range: {timesteps[0]} to {timesteps[-1]}')
print(f'Number of timesteps: {len(timesteps)}')

In [ ]:
# Extract columns
electricity_demand = filtered_data['P_Netz/MW'].to_numpy()
heat_demand = filtered_data['Q_Netz/MW'].to_numpy()
electricity_price = filtered_data['Strompr.€/MWh'].to_numpy()
gas_price = filtered_data['Gaspr.€/MWh'].to_numpy()

## Create TimeSeriesData Objects

`TimeSeriesData` allows specifying clustering weights and groups for aggregation:

In [ ]:
TS_heat_demand = fx.TimeSeriesData(heat_demand)
TS_electricity_demand = fx.TimeSeriesData(electricity_demand, clustering_weight=0.7)
TS_electricity_price_sell = fx.TimeSeriesData(-(electricity_price - 0.5), clustering_group='p_el')
TS_electricity_price_buy = fx.TimeSeriesData(electricity_price + 0.5, clustering_group='p_el')

## Build FlowSystem

In [ ]:
flow_system = fx.FlowSystem(timesteps)

# Add buses
flow_system.add_elements(
    fx.Bus('Strom', carrier='electricity', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Fernwärme', carrier='heat', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Gas', carrier='gas', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Kohle', carrier='fuel', imbalance_penalty_per_flow_hour=imbalance_penalty),
)

# Add effects
costs = fx.Effect('costs', '€', 'Kosten', is_standard=True, is_objective=True)
CO2 = fx.Effect('CO2', 'kg', 'CO2_e-Emissionen')
PE = fx.Effect('PE', 'kWh_PE', 'Primärenergie')

## Define Components

In [ ]:
# Gas Boiler
a_gaskessel = fx.linear_converters.Boiler(
    'Kessel',
    thermal_efficiency=0.85,
    thermal_flow=fx.Flow(label='Q_th', bus='Fernwärme'),
    fuel_flow=fx.Flow(
        label='Q_fu',
        bus='Gas',
        size=95,
        relative_minimum=12 / 95,
        previous_flow_rate=20,
        status_parameters=fx.StatusParameters(effects_per_startup=1000),
    ),
)

# Coal CHP
a_kwk = fx.linear_converters.CHP(
    'BHKW2',
    thermal_efficiency=0.58,
    electrical_efficiency=0.22,
    status_parameters=fx.StatusParameters(effects_per_startup=24000),
    electrical_flow=fx.Flow('P_el', bus='Strom', size=200),
    thermal_flow=fx.Flow('Q_th', bus='Fernwärme', size=200),
    fuel_flow=fx.Flow('Q_fu', bus='Kohle', size=288, relative_minimum=87 / 288, previous_flow_rate=100),
)

# Thermal Storage
a_speicher = fx.Storage(
    'Speicher',
    capacity_in_flow_hours=684,
    initial_charge_state=137,
    minimal_final_charge_state=137,
    maximal_final_charge_state=158,
    eta_charge=1,
    eta_discharge=1,
    relative_loss_per_hour=0.001,
    prevent_simultaneous_charge_and_discharge=True,
    charging=fx.Flow('Q_th_load', size=137, bus='Fernwärme'),
    discharging=fx.Flow('Q_th_unload', size=158, bus='Fernwärme'),
)

In [ ]:
# Sinks and Sources
a_waermelast = fx.Sink(
    'Wärmelast', inputs=[fx.Flow('Q_th_Last', bus='Fernwärme', size=1, fixed_relative_profile=TS_heat_demand)]
)

a_strom_last = fx.Sink(
    'Stromlast', inputs=[fx.Flow('P_el_Last', bus='Strom', size=1, fixed_relative_profile=TS_electricity_demand)]
)

a_gas_tarif = fx.Source(
    'Gastarif',
    outputs=[fx.Flow('Q_Gas', bus='Gas', size=1000, effects_per_flow_hour={costs.label: gas_price, CO2.label: 0.3})],
)

a_kohle_tarif = fx.Source(
    'Kohletarif',
    outputs=[fx.Flow('Q_Kohle', bus='Kohle', size=1000, effects_per_flow_hour={costs.label: 4.6, CO2.label: 0.3})],
)

a_strom_einspeisung = fx.Sink(
    'Einspeisung', inputs=[fx.Flow('P_el', bus='Strom', size=1000, effects_per_flow_hour=TS_electricity_price_sell)]
)

a_strom_tarif = fx.Source(
    'Stromtarif',
    outputs=[
        fx.Flow(
            'P_el',
            bus='Strom',
            size=1000,
            effects_per_flow_hour={costs.label: TS_electricity_price_buy, CO2.label: 0.3},
        )
    ],
)

In [ ]:
# Assemble FlowSystem
flow_system.add_elements(costs, CO2, PE)
flow_system.add_elements(
    a_gaskessel,
    a_waermelast,
    a_strom_last,
    a_gas_tarif,
    a_kohle_tarif,
    a_strom_einspeisung,
    a_strom_tarif,
    a_kwk,
    a_speicher,
)

# Visualize

flow_system.topology.plot('topology.html', show=False)

## Run Optimizations

### Full Optimization

Solve the entire problem at once:

In [ ]:
optimizations = []

if full:
    optimization = fx.Optimization('Full', flow_system.copy())
    optimization.do_modeling()
    optimization.solve(fx.solvers.HighsSolver(0.01 / 100, 60))
    optimizations.append(optimization)
    print('Full optimization completed')

### Segmented Optimization

Split the problem into overlapping time segments. Useful for:
- Very long time horizons
- Memory-constrained systems
- Rolling horizon approaches

In [ ]:
if segmented:
    optimization = fx.SegmentedOptimization('Segmented', flow_system.copy(), segment_length, overlap_length)
    optimization.do_modeling_and_solve(fx.solvers.HighsSolver(0.01 / 100, 60))
    optimizations.append(optimization)
    print('Segmented optimization completed')

### Aggregated (Clustered) Optimization

Reduce time series complexity through clustering. Useful for:
- Investment planning studies
- Screening large numbers of scenarios
- Problems where computational time is critical

In [ ]:
if aggregated:
    if keep_extreme_periods:
        clustering_parameters.time_series_for_high_peaks = [TS_heat_demand]
        clustering_parameters.time_series_for_low_peaks = [TS_electricity_demand, TS_heat_demand]

    optimization = fx.ClusteredOptimization('Aggregated', flow_system.copy(), clustering_parameters)
    optimization.do_modeling()
    optimization.solve(fx.solvers.HighsSolver(0.01 / 100, 60))
    optimizations.append(optimization)
    print('Aggregated optimization completed')

## Compare Results

Compare the solutions from different optimization modes:

In [ ]:
if len(optimizations) > 0:
    # Storage charge state comparison
    fx.plotting.with_plotly(
        get_solutions(optimizations, 'Speicher|charge_state'),
        mode='line',
        title='Charge State Comparison',
        ylabel='Charge state',
        xlabel='Time',
    )

In [ ]:
if len(optimizations) > 0:
    # CHP thermal output comparison
    fx.plotting.with_plotly(
        get_solutions(optimizations, 'BHKW2(Q_th)|flow_rate'),
        mode='line',
        title='BHKW2(Q_th) Flow Rate Comparison',
        ylabel='Flow rate',
        xlabel='Time',
    )

In [ ]:
if len(optimizations) > 0:
    # Operation costs over time
    fx.plotting.with_plotly(
        get_solutions(optimizations, 'costs(temporal)|per_timestep'),
        mode='line',
        title='Operation Cost Comparison',
        ylabel='Costs [€]',
        xlabel='Time',
    )

In [ ]:
if len(optimizations) > 0:
    # Total costs comparison
    fx.plotting.with_plotly(
        get_solutions(optimizations, 'costs(temporal)|per_timestep').sum('time'),
        mode='stacked_bar',
        title='Total Cost Comparison',
        ylabel='Costs [€]',
    ).update_layout(barmode='group')

In [ ]:
if len(optimizations) > 0:
    # Computation time comparison
    fx.plotting.with_plotly(
        pd.DataFrame(
            [calc.durations for calc in optimizations], index=[calc.name for calc in optimizations]
        ).to_xarray(),
        mode='stacked_bar',
    ).update_layout(title='Duration Comparison', xaxis_title='Optimization type', yaxis_title='Time (s)')

## Summary

| Mode | Best For | Trade-off |
|------|----------|----------|
| **Full** | Small-medium problems, exact solutions | Memory/time limited |
| **Segmented** | Long time horizons, rolling optimization | May lose global optimality |
| **Aggregated** | Investment planning, quick estimates | Approximates temporal dynamics |

Choose the mode based on your problem size and accuracy requirements!